In [26]:
import pandas as pd
import numpy as np

import time
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import confusion_matrix, recall_score, roc_auc_score

In [3]:
data = pd.read_csv('data_linear.csv')
data.shape

(1348099, 78)

In [4]:
pd.set_option('display.max_columns', None)
data.head()

,Unnamed: 0,int_rate,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,pub_rec,revol_bal,revol_util,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_bc_sats,num_bc_tl,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,issue_year,term_code,grade_code,sub_grade_code,emp_title_code,home_ownership_code,verification_status_code,purpose_code,title_code,earliest_cr_line_code,initial_list_status_code,last_pymnt_d_code,next_pymnt_d_code,last_credit_pull_d_code,application_type_code,disbursement_method_code,debt_settlement_flag_code,emp_length_code
0,0,13.99,55000.0,1,5.91,0.0,1.0,30.0,0.0,2765.0,29.7,122.67,564.0,560.0,0.0,30.0,0.0,722.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,2.0,5.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,2015,0.0,2.0,13.0,352908.0,1.0,0.0,2.0,16586.0,108.0,1.0,57.0,8.0,94.0,0.0,0.0,0.0,1.0
1,1,11.99,65000.0,1,16.06,1.0,4.0,6.0,0.0,21470.0,19.2,926.35,699.0,695.0,0.0,44.0,0.0,0.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,37.0,0.0,6.0,0.0,5.0,13.0,17.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,2015,0.0,2.0,10.0,103053.0,1.0,0.0,11.0,5247.0,165.0,1.0,77.0,8.0,94.0,0.0,0.0,0.0,1.0
2,2,10.78,63000.0,1,10.78,0.0,0.0,31.0,0.0,7869.0,56.2,15813.30,704.0,700.0,0.0,44.0,0.0,0.0,0.0,1.0,0.0,4.0,19.0,10827.0,73.0,0.0,2.0,2081.0,65.0,14000.0,2.0,5.0,1.0,6.0,31617.0,2737.0,55.9,0.0,0.0,125.0,184.0,14.0,14.0,5.0,101.0,37.0,10.0,33.0,0.0,2.0,2.0,4.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,2015,1.0,1.0,8.0,376427.0,1.0,0.0,4.0,16586.0,105.0,1.0,78.0,8.0,94.0,1.0,0.0,0.0,1.0
3,3,22.45,104433.0,1,25.37,1.0,3.0,12.0,0.0,21929.0,64.5,10128.96,704.0,700.0,0.0,44.0,0.0,0.0,1.0,3.0,0.0,3.0,14.0,73839.0,84.0,4.0,7.0,9702.0,78.0,34000.0,2.0,1.0,3.0,10.0,27644.0,4567.0,77.5,0.0,0.0,128.0,210.0,4.0,4.0,6.0,4.0,12.0,1.0,12.0,0.0,4.0,5.0,9.0,0.0,0.0,0.0,4.0,96.6,60.0,0.0,0.0,2015,1.0,5.0,25.0,67871.0,1.0,1.0,6.0,31101.0,414.0,1.0,66.0,8.0,93.0,0.0,0.0,0.0,3.0
4,4,13.44,34000.0,1,10.20,0.0,0.0,31.0,0.0,8822.0,68.4,7653.56,759.0,755.0,0.0,44.0,0.0,0.0,0.0,1.0,0.0,0.0,338.0,3976.0,99.0,0.0,0.0,4522.0,76.0,12900.0,0.0,0.0,0.0,0.0,2560.0,844.0,91.0,0.0,0.0,338.0,54.0,32.0,32.0,0.0,36.0,37.0,5.0,33.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,2015,0.0,2.0,12.0,317270.0,5.0,1.0,2.0,16586.0,650.0,1.0,101.0,8.0,104.0,0.0,0.0,0.0,4.0


In [5]:
data = data.drop(['Unnamed: 0'], axis = 1)
data.describe()

,int_rate,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,pub_rec,revol_bal,revol_util,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_bc_sats,num_bc_tl,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,issue_year,term_code,grade_code,sub_grade_code,emp_title_code,home_ownership_code,verification_status_code,purpose_code,title_code,earliest_cr_line_code,initial_list_status_code,last_pymnt_d_code,next_pymnt_d_code,last_credit_pull_d_code,application_type_code,disbursement_method_code,debt_settlement_flag_code,emp_length_code
count,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06
mean,1.324156e+01,7.623771e+04,8.001927e-01,1.827413e+01,3.176258e-01,6.621895e-01,3.263311e+01,2.150450e-01,1.627072e+04,5.181100e+01,5.415757e+03,6.772483e+02,6.601138e+02,1.709889e-02,4.392159e+01,5.038206e-03,2.363829e+02,1.020160e+00,2.316073e+00,3.096241e-01,1.296217e+00,1.555974e+01,2.820576e+04,7.180065e+01,1.167850e+00,2.400846e+00,4.861515e+03,5.807023e+01,3.240553e+04,1.037206e+00,6.417741e-01,1.523396e+00,4.667179e+00,1.316769e+04,9.961389e+03,5.995176e+01,9.043846e-03,1.486922e+01,1.261346e+02,1.805644e+02,1.284913e+01,7.758063e+00,1.645875e+00,2.335918e+01,3.762766e+01,6.488468e+00,3.393608e+01,4.830016e-01,3.609227e+00,4.702312e+00,8.043680e+00,7.469778e-04,3.240118e-03,8.424826e-02,2.169567e+00,9.446955e+01,4.479043e+01,1.342268e-01,5.202437e-02,2.014959e+03,2.410008e-01,1.748368e+00,1.070596e+01,1.999505e+05,2.912328e+00,1.008485e+00,2.696339e+00,1.979161e+04,3.923453e+02,5.815901e-01,7.198264e+01,8.083211e+00,8.110036e+01,1.914251e-02,5.118318e-03,2.469552e-02,3.456783e+00
std,4.765685e+00,6.992264e+04,3.998555e-01,1.115395e+01,8.777361e-01,9.600654e-01,1.552495e+01,6.014662e-01,2.247751e+04,2.452166e+01,7.114250e+03,8.139900e+01,1.328032e+02,1.467672e-01,1.095217e+01,7.710064e-02,1.077268e+04,7.637857e-01,1.927626e+00,7.334307e-01,1.124970e+00,1.582712e+01,2.780428e+04,1.345839e+01,1.015670e+00,1.775749e+00,3.499167e+03,1.322080e+01,3.571221e+04,1.002787e+00,1.950310e+00,1.729186e+00,3.129957e+00,1.591810e+04,1.501641e+04,2.760635e+01,1.096073e-01,8.087043e+02,5.010397e+01,9.209135e+01,1.595031e+01,8.508393e+00,1.967061e+00,3.007878e+01,1.109925e+01,5.483375e+00,1.302496e+01,1.295523e+00,2.193533e+00,2.891412e+00,4.680498e+00,2.910847e-02,6.070485e-02,4.911614e-01,1.796755e+00,8.611071e+00,3.517580e+01,3.776298e-

In [6]:
Target = data['loan_status']
Features = data.drop(['loan_status'], axis = 1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(Features, Target, test_size = 0.3, random_state = 42)

In [8]:
KF = KFold(3, shuffle = True, random_state = 42)

In [9]:
X1 = StandardScaler().fit_transform(X_train)
X2 = StandardScaler().fit_transform(X_test)

# Logistic Regression

In [10]:
start1 = time.time()
Logit = SGDClassifier(loss='log', shuffle= True, n_jobs = -1, 
                      warm_start = True, class_weight = "balanced").fit(X1, y_train)
end1 = time.time()
Time1 = end1 - start1
print(Time1)

7.19819188117981


In [29]:
train_scores1 = cross_val_score(Logit, X1, y_train, cv = KF)
test_scores1 = cross_val_score(Logit, X2, y_test, cv = KF)

train_ac1 = round(Logit.score(X1, y_train), 4)
test_ac1 = round(Logit.score(X2, y_test), 4)
recall1 = round(recall_score(y_test, Logit.predict(X2)), 4)
roc1 = round(roc_auc_score(y_test, Logit.predict_proba(X2)[:, 1]), 4)

print(train_scores1)
print(test_scores1)
print(train_ac1)
print(test_ac1)
print(recall1)
print(roc1)

[0.92938005 0.92985351 0.93156386]
[0.92990134 0.93068022 0.92947111]
0.931
0.9315
0.9287
0.9822


# SVM

In [15]:
from sklearn.kernel_approximation import Nystroem

X1_svm = Nystroem(random_state = 42, n_jobs = -1, n_components = 77, gamma = 1/77).fit_transform(X1)
X2_svm = Nystroem(random_state = 42, n_jobs = -1, n_components = 77, gamma = 1/77).fit_transform(X2)

In [34]:
from sklearn.calibration import CalibratedClassifierCV

start2 = time.time()
SVM = SGDClassifier(loss='hinge', shuffle= True, n_jobs = -1, random_state = 42, warm_start = True)
calibrated_svm = CalibratedClassifierCV(SVM, method='sigmoid').fit(X1_svm, y_train)
end2 = time.time()
Time2 = end2 - start2
print(Time2)

17.45238161087036


In [36]:
train_scores2 = cross_val_score(calibrated_svm, X1_svm, y_train, cv = KF)
test_scores2= cross_val_score(calibrated_svm, X2_svm, y_test, cv = KF)

train_ac2 = round(calibrated_svm.score(X1_svm, y_train), 4)
test_ac2 = round(calibrated_svm.score(X2_svm, y_test), 4)
recall2 = round(recall_score(y_test, calibrated_svm.predict(X2_svm)), 4)
roc2 = round(roc_auc_score(y_test, calibrated_svm.predict_proba(X2_svm)[:, 1]), 4)

print(train_scores2)
print(test_scores2)
print(train_ac2)
print(test_ac2)
print(recall2)
print(roc2)

[0.91417772 0.91420923 0.91423467]
[0.91216527 0.91265485 0.91190564]
0.9144
0.734
0.8888
0.5546


# Naive Bayes

In [18]:
start3 = time.time()
GNB = GaussianNB().fit(X1, y_train)
end3 = time.time()
Time3 = end3 - start3
print(Time3)

2.1127045154571533


In [32]:
train_scores3 = cross_val_score(GNB, X1, y_train, cv = KF)
test_scores3 = cross_val_score(GNB, X2, y_test, cv = KF)

train_ac3 = round(GNB.score(X1, y_train), 4)
test_ac3 = round(GNB.score(X2, y_test), 4)
recall3 = round(recall_score(y_test, GNB.predict(X2)), 4)
roc3 = round(roc_auc_score(y_test, GNB.predict_proba(X2)[:, 1]), 4)

print(train_scores3)
print(test_scores3)
print(train_ac3)
print(test_ac3)
print(recall3)
print(roc3)

[0.89851124 0.90899872 0.90724386]
[0.87685632 0.8760181  0.86802166]
0.9073
0.9069
0.961
0.9539


Save result as csv

In [40]:
df = pd.DataFrame([['Logit', train_ac1, test_ac1, recall1, roc1],
                   ['SVM', train_ac2, test_ac2, recall2, roc2],
                   ['Naive Bayes', train_ac3, test_ac3, recall3, roc3]], 
                    columns=['model', 'train score','test score','recall score', 'ROC AUC score'])
df

,model,train score,test score,recall score,ROC AUC score
0,Logit,0.9310,0.9315,0.9287,0.9822
1,SVM,0.9144,0.7340,0.8888,0.5546
2,Naive Bayes,0.9073,0.9069,0.9610,0.9539


In [41]:
df.to_csv('result.csv')